In [1]:
import warnings
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import scipy.sparse as sp
from scipy import stats

%matplotlib inline

warnings.filterwarnings('ignore')

DATA_PATH="./data/"

# Load Data

In [2]:
def load_txt(filename):
    feedbacks=[]
    with open(filename) as f:
        for line in f:
            feedbacks.append(line)
    return feedbacks

X_train=np.array(load_txt(f'{DATA_PATH}x_train.txt'))
X_test=np.array(load_txt(f'{DATA_PATH}x_test.txt'))
y_train=np.genfromtxt(f'{DATA_PATH}y_train.csv',delimiter=',',names=True)

In [3]:
len(X_train), len(X_test)

(3600000, 400000)

In [4]:
y_train.shape

(3600000,)

In [4]:
small_index=500000

In [5]:
small_X_train = X_train[:small_index]
small_y_train = y_train[:small_index]
small_X_train.shape, small_y_train.shape

((500000,), (500000,))

In [6]:
translator = str.maketrans('', '', string.punctuation)

In [7]:
test_X = np.copy(X_test)
train_X = np.copy(small_X_train)

for (x,), value in np.ndenumerate(train_X):
    train_X[x] = value.lower().translate(translator).replace('\n',' ').rstrip().lstrip()
for (x,), value in np.ndenumerate(test_X):
    test_X[x] = value.lower().translate(translator).replace('\n',' ').rstrip().lstrip()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import  CountVectorizer

def extract_features_cv(train_texts, test_texts, ngrams_count, stp_wrds=None, mindf=2, maxdf=0.9):
    tfidf = CountVectorizer(ngram_range=(1, ngrams_count), stop_words=stp_wrds, min_df=mindf, max_df=maxdf)
    tfidf.fit(np.concatenate((train_texts,test_texts)))
    train_features = tfidf.transform(train_texts)
    test_features = tfidf.transform(test_texts)
    return train_features, test_features

train_cv, test_cv = extract_features_cv(train_X,test_X, 2, 'english', 5, 0.9)

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
vectorizer.fit(np.concatenate((train_X,test_X)))
text_train_f = vectorizer.transform(train_X)
text_test_f = vectorizer.transform(test_X)

In [ ]:
train_f = sp.hstack((text_train_f,train_cv), format='csr')
test_f = sp.hstack((text_test_f,test_cv), format='csr')

In [ ]:
train_f.shape

In [ ]:
test_f.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1.0)

In [ ]:
clf.fit(train_f, small_y_train['Probability'])

In [ ]:
y_pred = clf.predict_proba(test_f)

In [ ]:
small_y_train

In [ ]:
with open('prediction.csv', 'w') as out:
    print('Id,Probability', file=out)
    for pair in enumerate(y_pred, 1):
        print('%i,%s' % pair, file=out)

In [ ]:
content.columns=['Id', 'Probability']

In [ ]:
content.to_csv('sol.csv',index=False)
